In [1]:
import ibis
import pandas as pd
from ibis import _
from ibis.backends.risingwave import Backend as RisingwaveBackend

con: RisingwaveBackend = RisingwaveBackend().connect(
                user="root",
                host="localhost",
                port=4566,
                database="dev")

df_table_l = pd.read_csv("./data/nullable_op/ints_strings.csv")
row = ibis.memtable(df_table_l.iloc[[0]])
row



OperationalError: connection to server at "localhost" (127.0.0.1), port 4566 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?


In [10]:
table_l = con.create("table_l", pd.read_csv("./data/nullable_op/ints_strings.csv"), overwrite=True)
table_r = con.create_table("table_r", pd.read_csv("./data/nullable_op/many_ints.csv"), overwrite=True)

query = (table_l
         .filter(_.int1 < 200)
         .mutate(mul=_.int1 * 20)
         .join(table_r
               .mutate(sum=_.int3 + 100), "int1")
               .select(["string1", "int1", "int3"]))

view = con.create_materialized_view("mat_view", query, overwrite=True)
view


InternalError_: Failed to run the query

Caused by:
  Permission denied: PermissionDenied: table used by 1 other objects.



In [11]:
table_l = con.create_table("table_l", table_l.union(row), overwrite=True)
view

InternalError_: Failed to run the query

Caused by:
  Permission denied: PermissionDenied: table used by 1 other objects.

